In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Research_dataset/Student_Review_data.tsv", sep="\t")
df.head()

,Unnamed: 0,StudentComments,Rating,totalwords,Sentiment,sent_pretrained,subjectivity,subj-score,isSame
0,0,good,4.96,1,positive,positive,subjective,0.6,true
1,1,good,5.00,1,positive,positive,subjective,0.6,true
2,2,teacher,4.25,1,positive,neutral,objective,0.0,fake
3,3,friendly teacher but not enough ability to enc...,4.38,10,positive,neutral,subjective,0.5,fake
4,4,teacher,4.92,1,positive,neutral,objective,0.0,fake


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2007747 entries, 0 to 2007746
Data columns (total 9 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0       int64  
 1   StudentComments  object 
 2   Rating           float64
 3   totalwords       int64  
 4   Sentiment        object 
 5   sent_pretrained  object 
 6   subjectivity     object 
 7   subj-score       float64
 8   isSame           object 
dtypes: float64(2), int64(2), object(5)
memory usage: 137.9+ MB


In [ ]:
df.isnull().sum()

,0
Unnamed: 0,0
StudentComments,0
Rating,0
totalwords,0
Sentiment,0
sent_pretrained,0
subjectivity,0
subj-score,0
isSame,0


In [ ]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df.shape

(2007747, 8)

In [ ]:
df_categorical=df.drop(['StudentComments','Rating','totalwords','subj-score'], axis=1)
df_categorical

,Sentiment,sent_pretrained,subjectivity,isSame
0,positive,positive,subjective,true
1,positive,positive,subjective,true
2,positive,neutral,objective,fake
3,positive,neutral,subjective,fake
4,positive,neutral,objective,fake
...,...,...,...,...
2007742,positive,positive,subjective,true
2007743,positive,positive,subjective,true
2007744,positive,positive,subjective,true
2007745,negative,neutral,subjective,fake


In [ ]:
for i in df_categorical.columns:  #Looping through unique value in every column to check
  print("----------------",i,"------------------\n")
  print(set(df_categorical[i].tolist()),"\n")


---------------- Sentiment ------------------

{'positive', 'neutral', 'negative'} 

---------------- sent_pretrained ------------------

{'positive', 'neutral', 'negative'} 

---------------- subjectivity ------------------

{'subjective', 'objective'} 

---------------- isSame ------------------

{'fake', 'true'} 



In [ ]:
import nltk
from nltk.util import pad_sequence
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

THis is a dataset for trail and error.!

In [ ]:
data = {
    'NumericFeature1': [10, 20, 15, 25,5.6,7.5,7,9,10,11],
    'NumericFeature2': [0.5, 0.7, 0.3, 0.8,2.2,4.5,10,99,999,10],
    'CategoricalFeature': ['A', 'B', 'A', 'C','Positive','Negative', 'bad','bad','bad','bad'],
    'StudentComments': ["Helo thi is a fake","This is a positive comment.","mr. manzurul sir is a good person, he teaches well","mam is goody for the cours","si is fin for the course","Hello sir.! this is Rafi Sir! sir", "Helo mam. THis is sonia Mamm", "Negative sentiment detected.", "Neutral feedback.", "Positive review."],
    'Fake/True' : ['fake', 'true', 'true','true','fake','fake','fake','fake','fake','fake']
}

df_new = pd.DataFrame(data)
df_new

In [ ]:
def clean_text_tf(text):
    if isinstance(text, str):
        # Remove non-alphabetic characters and URLs
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = re.sub('http\\S+', '', text)

        # Convert to lowercase
        text = text.lower()

        # Tokenize
        tokenizer = Tokenizer(num_words=500, oov_token='null')
        tokenizer.fit_on_texts([text])
        word_index= tokenizer.word_index
        tokens = tokenizer.texts_to_sequences([text])[0]
        # Apply stemming using NLTK
        #stemmer = SnowballStemmer(language='english')
        #tokens = [stemmer(language='english').stem(word) for word in tokens]

        # Apply lemmatization using NLTK
        #lemmatizer = WordNetLemmatizer()
        #tokens = [lemmatizer.lemmatize(word) for word in tokens]

        # Padding using TensorFlow
        padded_sequence = pad_sequences([tokens], maxlen=10, padding='post')[0]

        return padded_sequence
    else:
        return []

# Apply text processing to the 'StudentComments' column
df_new['ProcessedComments'] = df_new['StudentComments'].apply(clean_text_tf)

print(df_new[['StudentComments', 'ProcessedComments']])

                                     StudentComments  \
0                        This is a positive comment.   
1  mr. manzurul sir is a good person, he teaches ...   
2                         mam is goody for the cours   
3                           si is fin for the course   
4                       Negative sentiment detected.   
5                                  Neutral feedback.   
6                                   Positive review.   

                  ProcessedComments  
0    [2, 3, 4, 5, 6, 0, 0, 0, 0, 0]  
1  [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]  
2    [2, 3, 4, 5, 6, 7, 0, 0, 0, 0]  
3    [2, 3, 4, 5, 6, 7, 0, 0, 0, 0]  
4    [2, 3, 4, 0, 0, 0, 0, 0, 0, 0]  
5    [2, 3, 0, 0, 0, 0, 0, 0, 0, 0]  
6    [2, 3, 0, 0, 0, 0, 0, 0, 0, 0]  


Another way- Test 2
So far the approach below is the best preprocessing I got:
We can continue preprocessing without stemming and lemmatizing in the dataset. As we can get realationship from the textual words in number representaion of the sentences.


In [ ]:
tokenizer = Tokenizer()

# Text processing function using TensorFlow
def clean_text_tf(text):
    if isinstance(text, str):
        # Remove non-alphabetic characters and URLs
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = re.sub('http\\S+', '', text)

        # Convert to lowercase
        text = text.lower()
        #text = [word for word in text if word not in stopwords.words('english')]
        #text = [nltk.SnowballStemmer(language='english').stem(word) for word in text]
        #text = [nltk.WordNetLemmatizer().lemmatize(word) for word in text]


        return text
    else:
        return ""

# Apply text processing to the 'StudentComments' column
df['ProcessedComments'] = df['StudentComments'].apply(clean_text_tf)


# Fit Tokenizer on the entire 'StudentComments' column
tokenizer.fit_on_texts(df['ProcessedComments'])

# Tokenize and pad the sequences
df['ProcessedComments'] = tokenizer.texts_to_sequences(df['ProcessedComments'])
df['ProcessedComments'] = pad_sequences(df['ProcessedComments'], padding='post').tolist()

print(df[['StudentComments', 'ProcessedComments']])

                                           StudentComments  \
0                                                     good   
1                                                     good   
2                                                  teacher   
3        friendly teacher but not enough ability to enc...   
4                                                  teacher   
...                                                    ...   
2007742                                               good   
2007743                               he is a good teacher   
2007744                                  he is very good .   
2007745                                               okay   
2007746                                                yes   

                                         ProcessedComments  
0        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
2        [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
3        [3

In [ ]:
df.head()

,StudentComments,Rating,totalwords,Sentiment,sent_pretrained,subjectivity,subj-score,isSame,ProcessedComments
0,good,4.96,1,positive,positive,subjective,0.6,true,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,good,5.00,1,positive,positive,subjective,0.6,true,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,teacher,4.25,1,positive,neutral,objective,0.0,fake,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,friendly teacher but not enough ability to enc...,4.38,10,positive,neutral,subjective,0.5,fake,"[32, 3, 27, 18, 101, 263, 12, 414, 7, 49, 0, 0..."
4,teacher,4.92,1,positive,neutral,objective,0.0,fake,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
df.drop('StudentComments', axis=1, inplace=True)
df.shape

(2007747, 8)

In [ ]:
df.head()

,Rating,totalwords,Sentiment,sent_pretrained,subjectivity,subj-score,isSame,ProcessedComments
0,4.96,1,positive,positive,subjective,0.6,true,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,5.00,1,positive,positive,subjective,0.6,true,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,4.25,1,positive,neutral,objective,0.0,fake,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4.38,10,positive,neutral,subjective,0.5,fake,"[32, 3, 27, 18, 101, 263, 12, 414, 7, 49, 0, 0..."
4,4.92,1,positive,neutral,objective,0.0,fake,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


**change isSame column into numeric values**

In [ ]:
df['isSame'] = df['isSame'].apply(lambda x : 1 if x=='True' else 0)

***Changing the Categorical columns/features to numeric values by label encoding.***


In [ ]:
from sklearn.preprocessing import LabelEncoder
categories=['Sentiment', 'sent_pretrained','subjectivity']
labelEncoder= LabelEncoder()

for i in categories:
  df[i]= labelEncoder.fit_transform(df[i])


In [ ]:
df

,Rating,totalwords,Sentiment,sent_pretrained,subjectivity,subj-score,isSame,ProcessedComments
0,4.96,1,2,2,1,0.60,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,5.00,1,2,2,1,0.60,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,4.25,1,2,1,0,0.00,0,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4.38,10,2,1,1,0.50,0,"[32, 3, 27, 18, 101, 263, 12, 414, 7, 49, 0, 0..."
4,4.92,1,2,1,0,0.00,0,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...
2007742,4.15,1,2,2,1,0.60,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2007743,4.00,5,2,2,1,0.60,0,"[4, 2, 5, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2007744,4.00,5,2,2,1,0.78,0,"[4, 2, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2007745,2.30,1,0,1,1,0.50,0,"[250, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."




---




save dataset

---




In [ ]:
df.to_csv('/content/drive/MyDrive/Research_dataset/CleanDataset_Rafi1.csv')

In [ ]:
df['ProcessedComments'].dtype


dtype('O')

In [ ]:
df.dtypes

,0
Rating,float64
totalwords,int64
Sentiment,int64
sent_pretrained,int64
subjectivity,int64
subj-score,float64
isSame,int64
ProcessedComments,object


Model building ,evaluation

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'isSame' is your target variable
X = df.drop('isSame', axis=1)
y = df['isSame']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Test set shape: {X_test.shape}")


Training set shape: (1405422, 7)
Validation set shape: (301162, 7)
Test set shape: (301163, 7)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# Parameters
max_words = X_train['ProcessedComments'].apply(len).max()  # Ensure max_words is set correctly
vocab_size = len(tokenizer.word_index) + 1  # Total number of unique words in the tokenizer

# Model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128))  # No need for input_length here
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_5 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import numpy as np

# Convert lists to numpy arrays
X_train_sequences = np.array(X_train['ProcessedComments'].tolist())
X_val_sequences = np.array(X_val['ProcessedComments'].tolist())
X_test_sequences = np.array(X_test['ProcessedComments'].tolist())

# Ensure shapes are correct
print(X_train_sequences.shape)  # Should be (num_samples, max_sequence_length)
print(X_val_sequences.shape)
print(X_test_sequences.shape)


(1405422, 216)
(301162, 216)
(301163, 216)


In [ ]:
history = model.fit(
    X_train_sequences,
    y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_sequences, y_val)
)


NameError: name 'model' is not defined